In [ ]:
# data avaialble on kaggle:
# https://www.kaggle.com/kemical/kickstarter-projects

# following a course on feature engineering by Mat Leonard on kaggle:
# https://www.kaggle.com/learn/feature-engineering


In [3]:
# import modules 
import pandas as pd
import numpy as np

In [7]:
# load part of the dataset
ks_data = pd.read_csv('../data/raw/ks-projects-201801.csv', parse_dates=['deadline','launched'])

In [8]:
# inspect the data 
ks_data.head()

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,failed,0,GB,0.0,0.0,1533.95
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,failed,15,US,100.0,2421.0,30000.00
2,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,220.0,failed,3,US,220.0,220.0,45000.00
3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,1.0,failed,1,US,1.0,1.0,5000.00
4,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29,19500.0,2015-07-04 08:35:03,1283.0,canceled,14,US,1283.0,1283.0,19500.00


In [10]:
# check datatypes and missing values 
ks_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 378661 entries, 0 to 378660
Data columns (total 15 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   ID                378661 non-null  int64         
 1   name              378657 non-null  object        
 2   category          378661 non-null  object        
 3   main_category     378661 non-null  object        
 4   currency          378661 non-null  object        
 5   deadline          378661 non-null  datetime64[ns]
 6   goal              378661 non-null  float64       
 7   launched          378661 non-null  datetime64[ns]
 8   pledged           378661 non-null  float64       
 9   state             378661 non-null  object        
 10  backers           378661 non-null  int64         
 11  country           378661 non-null  object        
 12  usd pledged       374864 non-null  float64       
 13  usd_pledged_real  378661 non-null  float64       
 14  usd_

In [30]:
# create a copy of the dataset so that we don't need to reload it every time
ks = ks_data.copy()

## I. Baseline Model

#### Preparing target column

In [31]:
# prepare the classification variable (target column)
ks.state.value_counts()

failed        197719
successful    133956
canceled       38779
undefined       3562
live            2799
suspended       1846
Name: state, dtype: int64

In [32]:
# for the model we'll consider 1 -> successful and 0 -> otherwise
# ks['is_successful'] = (ks.state == 'successful').map({False:0, True:1})
# ks['is_successful'] = (ks['state'] == 'successful').astype(int)

# alternative approach using .assign()
ks = ks.assign(is_successful = (ks['state'] == 'successful').astype(int))

In [33]:
# drop live projects
ks = ks[~(ks['state'] == 'live')]

# alternative approach with .query()
# ks = ks.query('state != "live"')

#### Converting timestamps 

#### Processing categorical variables

#### Creating train and test sets

#### Training a RandomForest Classifier

#### Making predictions and scoring the model